In [8]:
from src.algorithms.alternating_least_squares import AlternatingLeastSquares
from src.helpers.dataset_indexer import DatasetIndexer
from src.helpers.checkpoint_manager import CheckpointManager
from src.recommenders import CollaborativeFilteringRecommenderBuilder
from src.backends import Backend
from src.helpers._logging import logger # noqa
from src.settings import settings

In [9]:
logger.info(
    "Log test"
)

2025-01-04 16:25:59,212 - INFO - Log test


In [10]:
dataset_indexer = DatasetIndexer(
    file_path="./ml-32m/ratings.csv",
    user_header="userId",
    item_header="movieId",
    rating_header="rating",
    limit=settings.general.LINES_COUNT_TO_READ,
)

indexed_data = dataset_indexer.index(
    approximate_train_ratio=settings.general.APPROXIMATE_TRAIN_RATIO
)

2025-01-04 16:25:59,276 - WARNING - The current implementation does not split the data into train and test sets exactly with the provided ratio. We use the provided ratio as a probability for a Bernoulli distribution to know whether the data point should be used as a training data or a test data.
2025-01-04 16:25:59,289 - WARNING - The limit of lines (.i.e 1000) to index has been reached. Exiting without loading the rest... 
2025-01-04 16:25:59,292 - INFO - Successfully indexed 1000 lines from ./ml-32m/ratings.csv


In [11]:
als_instance = AlternatingLeastSquares(
    hyper_lambda=settings.als.HYPER_LAMBDA,
    hyper_gamma=settings.als.HYPER_GAMMA,
    hyper_tau=settings.als.HYPER_TAU,
    hyper_n_epochs=settings.als.HYPER_N_EPOCH,
    hyper_n_factors=settings.als.HYPER_N_FACTOR,
)

als_backend = Backend(
    # Define the algorithm
    algorithm=als_instance,
    checkpoint_manager=CheckpointManager(
        checkpoint_folder=settings.als.CHECKPOINT_FOLDER,
        sub_folder=str(settings.general.LINES_COUNT_TO_READ),
    ),
    # Whether we should resume by using the last state of
    # the algorithm the checkpoint manager folder or not.
    resume=True,
)

In [12]:
recommender_builder = CollaborativeFilteringRecommenderBuilder(
    backend=als_backend,
)

# This might take some moment before finishing
recommender = recommender_builder.build(data=indexed_data)

2025-01-04 16:25:59,390 - INFO - Starting the build of the recommender using AlternatingLeastSquares...
2025-01-04 16:25:59,392 - INFO - Starting a model fitting using the backend AlternatingLeastSquares...
2025-01-04 16:25:59,394 - ERROR - Checkpoint list : []


TypeError: CheckpointManager.load() missing 1 required positional argument: 'checkpoint_name'

In [20]:
prediction_input = [{"rating": "4", "movieId": "17", "userId": "1"}]
recommender.recommend(prediction_input)

predictions => [3.67311552 3.2774162  3.56581707 ... 3.94138817 3.48768609 3.94138784]
items_ids => [17521 17512 17330 ... 17646 23931 13676]
predictions => [3.67311552 3.2774162  3.56581707 ... 3.94138817 3.48768609 3.94138784]
items_ids => [17521 17512 17330 ... 17646 23931 13676]
Recommending [17521 17512 17330 ... 17646 23931 13676]


array([3.67311552, 3.2774162 , 3.56581707, ..., 3.94138817, 3.48768609,
       3.94138784], shape=(26505,))